In [49]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import re
from dateutil.parser import parse

In [50]:
years = []
for i in range(2, 24):
    year = '20'
    if(i < 10): 
        year += '0'
    year += str(i)
    years.append(year)
print(years)   

['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [51]:
file = './'+years[3]+'.csv'
df = pd.read_csv(file)
df.head()

,Unnamed: 0,index,Date,NERC,Event,Area,Disturbance,Loss,Affected Customers,Restoration Date
0,0,3,2005-01-04 00:00:00,SPP,6:00 p.m.,Eastern one third of the state of Kansas,Winter Storm,200,211000,"1/14/05, 12:00 p.m."
1,1,4,2005-01-05 00:00:00,ECAR,4:00 p.m.,Akron and Mansfield areas,Ice Storm,250,246990,"1/13/05, 6:00 p.m."
2,2,5,2005-01-05 00:00:00,ECAR,9:10 p.m.,Indiana Michigan Region - Muncie District,Winter Ice Storm,545,114791,"1/16/05, 11:00 a.m."
3,3,6,2005-01-07 00:00:00,WECC,1:00 p.m.,Northern California,Winter Storm,120,442000,"1/10/05, 8:00 a.m."
4,4,8,2005-01-23 00:00:00,PR,10:42 a.m.,Island of Puerto Rico,Voltage Reduction,140,NaN,"1/23/05, 11:24 a.m."


In [57]:
def fstCleaning(df, name):
  digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
  for index, i in df.iterrows():
    if(i['Restoration Date'][0] in digits):
      date_string = i['Restoration Date']
      #date_string = re.sub(r"/", "/", date_string)
      date_string = date_string.split(",")[0]
      datetime_object = datetime.strptime(date_string, '%m/%d/%y')
      df.loc[index, 'Restoration Date'] = datetime_object
    else:
      datetime_object = i['Date']
      df.loc[index, 'Restoration Date'] = datetime_object
  outFile = name + '.csv'
  df.to_csv(outFile, sep=',', encoding='utf-8')

In [53]:
def sndCleaning(df, year):
  digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
  for index, i in df.iterrows():
    if(i['Restoration Date'][0] in digits):
      parsed_datetime = parse(f"{i['Restoration Date']}, {year}")
      df.loc[index, 'Restoration Date'] = parsed_datetime
    else: 
      converted_string = i['Date']
      df.loc[index, 'Restoration Date'] = converted_string
  outFile = year + '.csv'
  df.to_csv(outFile, sep=',', encoding='utf-8')

In [54]:
def is_format1(date_str):
  return len(date_str) == 10 and date_str.count("/") == 2

def convert_format1(date_str):
  date_obj = datetime.strptime(date_str, "%m/%d/%Y")
  return date_obj.strftime("%Y-%m-%d 00:00:00")

def detect_and_convert(date_str):
  if is_format1(date_str):
    converted_date = convert_format1(date_str)
    format_detected = True
  else:
    converted_date = date_str
    format_detected = False
  return converted_date, format_detected


In [55]:
def trdCleaning(df, year):
    for index, i in df.iterrows():
        new_data, dump = detect_and_convert(i['Restoration Date'])
        df.loc[index, 'Restoration Date'] = new_data
        outFile = year + '.csv'
        df.to_csv(outFile, sep=',', encoding='utf-8')

def trdexCleaning(df, year):
    for index, i in df.iterrows():
        new_data, dump = detect_and_convert(i['Date'])
        df.loc[index, 'Date'] = new_data
        outFile = year + '.csv'
        df.to_csv(outFile, sep=',', encoding='utf-8')

In [59]:
year_data = {}
for year in years:
    file = year + '.csv'
    year_df = pd.read_csv(file)
    year_data[year] = year_df
i = 0
for year in years:
    print(i)
    if(i>=3 and i < 4):
        fstCleaning(year_data[year], year)
    elif(i >= 4 and i < 9):
        sndCleaning(year_data[year], year)
    if(i>=14 and i < 22):
        trdexCleaning(year_data[year], year)
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [60]:
year_data = {}
for year in years:
    file = year + '.csv'
    year_df = pd.read_csv(file)
    year_data[year] = year_df
i = 0
for year in years:
    labelsDrop = []
    for column in year_data[year].columns:
        if('Unnamed' in column or 'index' in column):
            labelsDrop.append(column)
    year_data[year].drop(columns=labelsDrop, inplace=True)
    outFile = year + '.csv'
    year_data[year].to_csv(outFile, sep=',', encoding='utf-8')

combined_df = pd.concat(year_data) 
combined_df